In [1]:
import pandas as pd

In [2]:
wiki_df = pd.read_json('../data/wikipedia_documents.json', orient='index')
len(wiki_df['text'].unique())

56737

In [3]:
texts = []
titles = []

for i in range(len(wiki_df)):
    wiki_context = wiki_df['text'][i]
    wiki_title = wiki_df['title'][i]

    if wiki_title in wiki_context:
        texts.append(wiki_context)
        titles.append(wiki_title)

wiki_qa_df = pd.DataFrame(data={'text':texts,'title':titles})
wiki_qa_df.head()

,text,title
0,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",나라 목록
1,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,백남준
2,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",아오조라 문고
3,아오조라 문고는 자원봉사로 운영되며 열람 역시 무료이다. 서비스 개시 초반에는 보...,아오조라 문고
4,"텍스트 파일을 아오조라 문고에 수록할 때, 텍스트 파일이 갖추어야 할 서식을 '아오...",아오조라 문고


In [6]:
import pandas as pd
wiki_qa_df = pd.read_csv('wiki_text_title.csv')

In [15]:
for i in [2233, 2234, 2235]:
    print(wiki_qa_df.iloc[i].text)

안나푸르나산(Mount Annapurna, अन्नपूर्णा)는 히말라야산맥 중부에 줄지어선 고봉이다. 길이가 무려 55km에 달하고, 최고봉인 안나푸르나 1봉은 높이가 8,091m로 8,000m이상의 고산을 의미하는 14좌의 하나이다. 서쪽에서부터 최고봉인 안나푸르나 1봉(8,091m), 안나푸르나 3봉(7,555m), 안나푸르나 4봉(7,525m), 안나푸르나 2봉(7,937m), 강가푸르나(7,455m)가 연이어 서있고, 안나푸르나 3봉의 남쪽에서 갈라져 나온 끝에 마차푸차레산가 있다. 안나푸르나는 산스크리트어로 '수확의 여신'이라는 뜻을 가지고 있다.
안나푸르나산에는 7,200m를 넘는 여섯 봉우리가 있다.
{| border=0
! 이름 !! 높이 !! 상대 높이 !! 좌표
|-
|안나푸르나 1봉  ||8,091m  || 2,984m ||28.595|N|83.819|E|type:mountain_region:NP|name=안나푸르나 1봉
|-
|안나푸르나 2봉  ||7,937m  || 2,437m ||28.539|N|84.137|E|type:mountain_region:NP|name=안나푸르나 2봉
|-
|안나푸르나 3봉  ||7,555m  || 703m ||28.585|N|84.000|E|type:mountain_region:NP|name=안나푸르나 3봉
|-
|안나푸르나 4봉  ||7,525m || ||28.539|N|84.087|E|type:mountain_region:NP|name=안나푸르나 4봉
|-
|강가푸르나      ||7,455m   || 563m ||28.606|N|83.965|E|type:mountain_region:NP|name=강가푸르나
|-
|안나푸르나 남봉 ||7,219m  || 775m ||28.518|N|83.806|E|type:mountain_region:NP|name=안나푸르나 남봉
|}
1950년 6월 3일 프랑스의 모리스 에르조그와 루이 라슈날이 처음으로 안나푸르나 제1봉 등정에 성공하였다. 이는 인류 최초의 8,0

In [6]:
wiki_qa_df.to_csv('wiki_text_title.csv')

In [1]:
import sys
# sys.path.append("..")
import random
from argparse import ArgumentParser

import torch
from tokenizers import SentencePieceBPETokenizer
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import GPT2LMHeadModel

from korquad_qg.config import QGConfig
from korquad_qg.dataset import (MAX_QUESTION_SPACE, MIN_QUESTION_SPACE, QGDecodingDataset, load_wiki_dataset)

model = GPT2LMHeadModel.from_pretrained("taeminlee/kogpt2")
model.load_state_dict(torch.load('./models/qg_kogpt2/QG_kogpt2.pth')) # map_location="cpu"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

tokenizer = SentencePieceBPETokenizer.from_file(
    vocab_filename="./korquad_qg/tokenizer/vocab.json", merges_filename="./korquad_qg/tokenizer/merges.txt", add_prefix_space=False
)

examples = load_wiki_dataset('./wiki_text_title.csv')
random.shuffle(examples)
dataset = QGDecodingDataset(examples, tokenizer, 512)
dataloader = DataLoader(dataset, batch_size=1)

model = model.to(device)
model.eval()

generated_results = []

for i, batch in tqdm(enumerate(dataloader), desc="generate", total=len(dataloader)):
    input_ids, attention_mask = (v.to(device) for v in batch)
    origin_seq_len = input_ids.size(-1)

    decoded_sequences = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=origin_seq_len + MAX_QUESTION_SPACE,
        min_length=origin_seq_len + MIN_QUESTION_SPACE,
        pad_token_id=0,
        bos_token_id=1,
        eos_token_id=2,
        num_beams=5,
        repetition_penalty=1.3,
        no_repeat_ngram_size=3,
        num_return_sequences=1,
    )

    for decoded_tokens in decoded_sequences.tolist():
        decoded_question_text = tokenizer.decode(decoded_tokens[origin_seq_len:])
        decoded_question_text = decoded_question_text.split("</s>")[0].replace("<s>", "")
        generated_results.append(
            (examples[i].context, examples[i].answer, examples[i].question, decoded_question_text)
        )
'''
with open('Question_generation.csv', "w") as f:
    for context, answer, question, generated_question in generated_results:
        f.write(f"{generated_question}\t{answer}\t{context}\n")
'''

generate: 100%|██████████| 28765/28765 [4:39:00<00:00,  1.72it/s]  


In [3]:
generated_results[:5]

[('안나푸르나산에는 7,200m를 넘는 여섯 봉우리가 있다.\n{| border=0\n! 이름 !! 높이 !! 상대 높이 !! 좌표\n|-\n|안나푸르나 1봉  ||8,091m  || 2,984m ||28.595|N|83.819|E|type:mountain_region:NP|name=안나푸르나 1봉\n|-\n|안나푸르나 2봉  ||7,937m  || 2,437m ||28.539|N|84.137|E|type:mountain_region:NP|name=안나푸르나 2봉\n|-\n|안나푸르나 3봉  ||7,555m  || 703m ||28.585|N|84.000|E|type:mountain_region:NP|name=안나푸르나 3봉\n|-\n|안나푸르나 4봉  ||7,525m || ||28.539|N|84.087|E|type:mountain_region:NP|name=안나푸르나 4봉\n|-\n|강가푸르나      ||7,455m   || 563m ||28.606|N|83.965|E|type:mountain_region:NP|name=강가푸르나\n|-\n|안나푸르나 남봉 ||7,219m  || 775m ||28.518|N|83.806|E|type:mountain_region:NP|name=안나푸르나 남봉\n|}',
  '안나푸르나산',
  None,
  '무르시엘라고 불리는 산은?'),
 ('19 세기까지 고대 이집트 의학에 대한 주요 정보 출처는 고대 후기의 글이었다.\xa0그리스 역사가인\xa0헤로도토스는\xa0기원전 440년경에 이집트를 방문하여 그들의 의학적 관행에 대한 그의 관찰을 광범위하게 기록했다.\xa0\xa0Pliny the Elder 는 또한 역사적인 검토에서 그들에 호의적으로 썼다.  Hippocrates (약의 아버지), Herophilos, Erasistratus 그리고 나중에 Galen 은 아멘 호프 (Amenhotep ) 성전에서 공부했으며 고대 이집트 의학이 그리스 의학에 기여한 것을 인정했다 .\n\n1822 년 로제타 스톤 (Roset

In [17]:
context_list = []
question_list = []
answers_list = []
for context, answer, _, generated_question in generated_results:
    context_list.append(context)
    answers_list.append(answer)
    question_list.append(generated_question)
    
generated_dict = {'context':context_list,
                 'question':question_list,
                 'answers':answers_list}

In [23]:
pd.DataFrame(generated_dict).to_csv('question_generation.csv')